<a href="https://colab.research.google.com/github/AmanKabra/COVID-testing-capacity/blob/master/SupplyAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Public Labs calculation

---



In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import geopandas as gpd
import plotly.express as px

In [ ]:
df1=pd.read_excel("Parameters.xlsx",sheet_name="PublicLabs")
df1=df1.set_index('State')

In [ ]:
publiclabs=pd.DataFrame(index=df1.index)
publiclabs

In [ ]:
for i in range(1,201):
  publiclabs[i]=df1[0].astype(int)+((i/df1["Frequency of addition (days)"].astype(int)).apply(np.floor)*df1["Number of labs"].astype(int))
publiclabs=publiclabs.astype(int)

publiclabs.insert(0,0,df1[0])


In [ ]:
parameters = pd.read_excel('Parameters.xlsx',sheet_name='Parameters') 
parameters

In [ ]:
publiccap=publiclabs*parameters.iloc[9,1]*parameters.iloc[10,1]

Private labs calculations










---



In [ ]:
GeneXvalue = (parameters.loc[5][1]/parameters.loc[1][1]*parameters.loc[0][1]*(1-parameters.loc[4][1])*(1-parameters.loc[2][1])).astype(int)*939*parameters.loc[6][1]/30
GeneXvalue=GeneXvalue.astype(int)

genex = pd.Series([])

i=0
while i<201:
  if i<parameters.iloc[3,1]:
    genex[i]=0
  else:
    genex[i]=GeneXvalue
  i+=1

genex

In [ ]:
abottvalue = (parameters.loc[5][2]/parameters.loc[1][2]*parameters.loc[0][2]*(1-parameters.loc[4][2])*(1-parameters.loc[2][2])).astype(int)*11*parameters.loc[6][2]/30
abottvalue = abottvalue.astype(int)

abott = pd.Series([])


i=0
while i<201:
  if i<parameters.iloc[3,2]:
    abott[i]=0
  else:
    abott[i]=abottvalue
  i+=1


In [ ]:
rawdata = pd.read_excel('Parameters.xlsx',sheet_name='RawData') 
rawdata[:] = rawdata[:].replace({'Unknown':np.nan})
ICMRapproved = rawdata[rawdata['ICMR Approved']==1]

In [ ]:
totalrtpcr=rawdata['qPCR instruments'].sum()

In [ ]:

rtpcrvalue = (parameters.loc[5][3]/parameters.loc[1][3]*parameters.loc[0][3]*(1-parameters.loc[4][3])*(1-parameters.loc[2][3])).astype(int)*(int(totalrtpcr)+150)*parameters.loc[6][3]/30
rtpcrvalue = rtpcrvalue.astype(int)

rtpcr = pd.Series([])


i=0
while i<201:
  if i<parameters.iloc[3,3]:
    rtpcr[i]=0
  else:
    rtpcr[i]=rtpcrvalue
  i+=1

rtpcr

In [ ]:
try1 = pd.read_excel('Parameters.xlsx',sheet_name='RTPCR Capacity')
RTPCRnewkits = try1.iloc[:,0]
type(RTPCRnewkits)
RTPCRnewkits

In [ ]:
Dailykitsallocated = np.minimum(rtpcr,RTPCRnewkits)
Dailykitsallocated

In [ ]:
CarriedOverTestKits=pd.Series()

In [ ]:
CarriedOverTestKits=RTPCRnewkits-Dailykitsallocated
CarriedOverTestKits.describe()
CumSumCarriedOverTestKits=CarriedOverTestKits.cumsum()
CumSumCarriedOverTestKits

In [ ]:
PCRTestKitsAvailableStock=CumSumCarriedOverTestKits+RTPCRnewkits
PCRTestKitsAvailableStock

In [ ]:
DTCManualActualCapacity = np.minimum(PCRTestKitsAvailableStock,rtpcr)
DTCManualActualCapacity

In [ ]:
TotalTheoreticalCapacity = rtpcr+abott+genex
TotalTheoreticalCapacity

In [ ]:
TotalActualCapacity = abott+genex+DTCManualActualCapacity
TotalActualCapacity

In [ ]:
totalICMRrtpcr=ICMRapproved['qPCR instruments'].sum()
f45 = totalICMRrtpcr*(parameters.loc[5][3]/parameters.loc[1][3]*parameters.loc[0][3]*(1-parameters.loc[4][3])*(1-parameters.loc[2][3])).astype(int)
df5 = pd.Series(range(0, 201 ,1))
df6 = (df5/7).astype(int)
df6

In [ ]:
parameters

In [ ]:
#RTPCR='Private Labs- FINAL'!$F$45+ ROUNDDOWN(A3/7,0)*'Private Labs- FINAL'!$F$52


F45=ICMRapproved["qPCR instruments"].sum().astype(int)*(parameters.loc[5][3]/parameters.loc[1][3]*parameters.loc[0][3]*(1-parameters.loc[4][3])*(1-parameters.loc[2][3])).astype(int)
F45
F52=parameters.iloc[7,3]*math.floor(rawdata["qPCR instruments"].mean())*(parameters.loc[5][3]/parameters.loc[1][3]*parameters.loc[0][3]*(1-parameters.loc[4][3])*(1-parameters.loc[2][3])).astype(int)*7
F52
new9=df6*F52+F45

In [ ]:

genex2 = pd.Series([])

d52=parameters.iloc[7,1]*int(rawdata['GeneXpert Modules'].mean())*(parameters.loc[5][1]/parameters.loc[1][1]*parameters.loc[0][1]*(1-parameters.loc[4][1])*(1-parameters.loc[2][1])).astype(int)*7

i=0
while i<201:
  if i<parameters.iloc[3,1]:
    genex2[i]=0
  else:
    genex2[i]=((parameters.loc[5][1]/parameters.loc[1][1]*parameters.loc[0][1]*(1-parameters.loc[4][1])*(1-parameters.loc[2][1])).astype(int)*ICMRapproved['GeneXpert Modules'].sum())+(df6[i]*d52)
  i+=1

new10=new9+genex2
new10

In [ ]:
ICMRTheoreticalCapacity=np.minimum(new10, TotalTheoreticalCapacity)
ICMRTheoreticalCapacity

In [ ]:
ICMRActualCapacity=np.minimum(ICMRTheoreticalCapacity, TotalActualCapacity)
ICMRActualCapacity

In [ ]:
CumulativeTestsActual = genex+abott+DTCManualActualCapacity
CumSumCumulativeTestsActual=CumulativeTestsActual.cumsum()
CumSumCumulativeTestsActual

In [ ]:
CumSumICMRActualCapacity=ICMRActualCapacity.cumsum()
CumSumICMRActualCapacity

**Day wise calculations**

In [ ]:
df10=pd.read_excel("Parameters.xlsx",sheet_name="PrivateLabs")
df10=df10.set_index('State')

privatelabs=pd.DataFrame(index=df10.index)

i=0
for i in range(1,201):
  privatelabs[i]=df10[0].astype(int)+((i/df10["Frequency of addition (days)"].astype(int)).apply(np.floor)*df10["Number of labs"].astype(int))
privatelabs=privatelabs.astype(int)
privatelabs

privatelabs.insert(0,0,df10[0])

privatelabs[191]



In [ ]:
sum = privatelabs.sum()
prop = pd.DataFrame()
for i in range(0,201):
  prop[i]=privatelabs[i]/sum[i]

**Private Theoretical Capacity**

In [ ]:
finalprivatetheoreticalcapacity = pd.DataFrame()

for i in range (0,201):
  finalprivatetheoreticalcapacity[i] = prop[i]*TotalTheoreticalCapacity[i]

finalprivatetheoreticalcapacity

Summing up total capacity

In [ ]:
totalcapacity = finalprivatetheoreticalcapacity + publiccap

In [ ]:
totalcapacity=totalcapacity.astype(int)
totalcapacity
cumsum = np.cumsum(totalcapacity, axis=1)

In [ ]:
plt.plot(cumsum.loc["Orissa"])
plt.xlabel("Number of days from today")
plt.ylabel("Cumulative theoretical testing capacity (orissa)")
plt.grid(True)
plt.tight_layout()
plt.savefig("graph.pdf")


**Plotting Map**

In [ ]:
SortedDF = cumsum.drop('Chennai')
SortedDF = SortedDF.rename(index={'Andaman and Nicobar Islands': 'Andaman and Nicobar'})
SortedDF = SortedDF.rename(index={'Jammu and Kashmir': 'Jammu and Kashmir'})
SortedDF = SortedDF.append(pd.Series(name='Daman and Diu'))
SortedDF = SortedDF.append(pd.Series(name='Lakshadweep'))
SortedDF.sort_index(inplace=True)
SortedDF = SortedDF.fillna(0)
SortedDF

In [ ]:
shapefile = gpd.read_file("IND_adm1.shp")

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


def f(x):
  newcoloumn = SortedDF[int(x)]
  newcoloumn.reset_index(drop=True, inplace=True)
  global shapefile
  shapefile = shapefile.assign(capacity=newcoloumn)
  variable = 'capacity'
  vmin, vmax = newcoloumn.min(), newcoloumn.max()
  fig, ax = plt.subplots(1, figsize=(10, 6))
  ax.axis('off')
  ax.set_title('Testing capacity for COVID-19', fontdict={'fontsize': '20', 'fontweight': '3'})
  ax.annotate('collaboration with CHAI (2020)',xy=(0.1, .08),  xycoords='figure fraction', horizontalalignment='center', verticalalignment='top', fontsize=12, color='#555555')
  sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
  sm._A = []
  cbar = fig.colorbar(sm)
  capacityplot = shapefile.plot(column=variable, cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')
  fig.savefig("map_export.png", dpi=200)


interact(f, x=widgets.IntSlider(min=0, max=200, step=1, value=10))